# **라이브러리 불러오기**


In [ ]:
!pip install selenium
!pip install webdriver_manager.chrome
!pip install pororo
!pip install soynlp.normalizer
!pip install webdriver-manager
!pip install soynlp

  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


ERROR: Could not find a version that satisfies the requirement webdriver_manager.chrome (from versions: none)
ERROR: No matching distribution found for webdriver_manager.chrome
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.3 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


ERROR: Could not find a version that satisfies the requirement soynlp.normalizer (from versions: none)
ERROR: No matching distribution found for soynlp.normalizer
     |████████████████████████████████| 416 kB 24.0 MB/s 


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests

from selenium import webdriver
import time
from openpyxl import Workbook
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

from pororo import Pororo
import sys
sys.path.insert(0, '../')
import soynlp
from soynlp.normalizer import *

print(soynlp.__version__)

0.0.493


# 2021-8~2022-2 트게더 크롤링

In [ ]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}

title_list = []
date_list = []

pagenum = 
pagenum2 = 

for i in range(pagenum, pagenum2): # range 내 pagenum, pagenum2 은 트게더 게시판 직접 확인 후 21년 8월부터 22년 2월 사이 page 로 설정
    url = "https://tgd.kr/s/셀럽주소/page/" + str(i)

    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")

    articles = soup.find("div", id="article-list")
    items = articles.find_all("div", attrs={"class":"item"})
    for item in items:
        title = item.find("a").get_text().strip()
        print(title)
        title_list.append(title)
        date = item.find("div", attrs={"class":"list-time"}).get_text().strip()
        print(date)
        date_list.append(date)

pd_data = {"셀럽명": "셀럽병", "제목" : title_list , "날짜" : date_list}
tgd_pd = pd.DataFrame(pd_data)

tgd_pd.to_csv("셀럽명.csv", index=False, encoding='utf-8-sig')

# **전처리 (셀럽 별 csv 병합 과정 생략)**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/fancim_tgd.csv') # 팬심을 사용하는 셀럽의 트게더
df1 = pd.read_csv('/content/drive/MyDrive/nonfancim_tgd.csv') # 팬심을 사용하지 않는 셀럽의 트게더

In [ ]:
df = df.drop(df[df['제목'].str.contains('일정')].index) # 137개
df = df.drop(df[df['제목'].str.contains('공지')].index)# 217개
df = df.drop(df[df['제목'] == '오늘'].index) # 48개

In [ ]:
df1 = df1.drop(df[df['제목'].str.contains('일정')].index) # 1개
df1 = df1.drop(df[df['제목'].str.contains('공지')].index)# 6개
df1 = df1.drop(df[df['제목'] == '오늘'].index) # 2개

In [ ]:
df['제목'] = df['제목'].apply(repeat_normalize)
df['제목'] = df['제목'].apply(only_hangle)

stop_words = ['ㄱ','ㄴ','ㄷ','ㄹ','ㅁ','ㅂ','ㅅ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ',
              'ㅏ','ㅑ','ㅓ','ㅕ','ㅜ','ㅠ','ㅡ','ㅣ',
              'ㄱㄱ','ㄴㄴ','ㄷㄷ','ㄹㄹ','ㅁㅁ','ㅂㅂ','ㅅㅅ','ㅇㅇ','ㅈㅈ','ㅊㅊ','ㅋㅋ','ㅌㅌ','ㅍㅍ','ㅎㅎ',
              'ㅏㅏ','ㅑㅑ','ㅓㅓ','ㅕㅕ','ㅜㅜ','ㅠㅠ','ㅡㅡ','ㅣㅣ']

for w in stop_words:
  df['제목'] = df['제목'].replace(w,'', regex=True)

In [ ]:
df1['제목'] = df1['제목'].apply(repeat_normalize)
df1['제목'] = df1['제목'].apply(only_hangle)

for w in stop_words:
  df1['제목'] = df1['제목'].replace(w,'', regex=True)

# **감성분석**

## 팬심을 사용하는 데이터 전처리 및 감성분석

In [ ]:
# Pororo 라이브러리 불러오기
sa = Pororo(task="sentiment", model="brainbert.base.ko.nsmc", lang="ko")
sh = Pororo(task="sentiment", model="brainbert.base.ko.shopping", lang="ko")

In [ ]:
# 0.7이상일때 부정적 판단을 위한 함수
def check_neg(dic):
    if dic['negative'] >= 0.7:
        return 'negative'
    else:
        return 'positive'

In [ ]:
# 감성분석 라이브러리 적용
df['labels_ns'] = df['댓글'].apply(sa, show_probs=True)
df['labels_sh'] = df['댓글'].apply(sh, show_probs=True)

df['labels_ns_threshold'] = df['labels_ns'].apply(check_neg)
df['labels_sh_threshold'] = df['labels_sh'].apply(check_neg)
df['both_labels'] = 'positive'
df.loc[(df['labels_ns_threshold']=='negative')&(df['labels_sh_threshold']=='negative'), 'both_labels'] = 'negative'

In [ ]:
# 사용할 컬럼만 남기기
df = df[['셀럽명', '제목', '날짜', 'both_labels']]

In [ ]:
# 확인 및 보관용 csv 파일저장
df.to_csv('fancim_tgd_labels.csv', index=False, encoding='utf-8-sig')

## 팬심을 사용하지 않는 데이터 전처리 및 감성분석

In [ ]:
# 감성분석 라이브러리 적용
df1['labels_ns'] = df1['댓글'].apply(sa, show_probs=True)
df1['labels_sh'] = df1['댓글'].apply(sh, show_probs=True)

df1['labels_ns_threshold'] = df1['labels_ns'].apply(check_neg)
df1['labels_sh_threshold'] = df1['labels_sh'].apply(check_neg)
df1['both_labels'] = 'positive'
df1.loc[(df1['labels_ns_threshold']=='negative')&(df1['labels_sh_threshold']=='negative'), 'both_labels'] = 'negative'

In [ ]:
# 사용할 컬럼만 남기기
df1 = df1[['셀럽명','제목','날짜','both_labels']]

In [ ]:
# 확인 및 보관용 csv 파일저장
df1.to_csv('non_fancim_tgd_labels.csv', index=False, encoding='utf-8-sig')